# Dashboarding tools

In [1]:
%matplotlib notebook

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt, os, re
from time import time
import warnings

from pprint import pprint

from IPython.display import display
from ipywidgets import widgets, interactive, HBox, VBox, Layout
from pandas.api.types import CategoricalDtype
from matplotlib.ticker import FuncFormatter

sns.set_style('white')
plt.rcParams["figure.figsize"] = (15,8)

In [2]:
# Data files location
DATA_DIR = "export/data/"

# Smallest granularity
SAMPLING = "D"

# Size column
SIZE_COLUMN = "Horizontal Component Code"

# Default dates for dropdowns
START_DATE, END_DATE =  dt.date(2015,8,1), dt.date(2017,12,31)

# Filters
FILTERS = [
        
        ("Web Shop Code", "Retailer"), 
        ("Merchandise Code", "Category"),
        ("Brand", "Brand"),
        ("Season", "Season"),
        ("Item No_", "Item Number"),
#         ("External Record Id", "Product ID")   

]

In [3]:
def load_sales(nrows=None, **csv_params):
    
    filename = join_path("ExportSalesData_3_0_stripped.csv")
    exclude = ['Kosten verzending', 'AFTERPAY (WEB) Nerderland']
    
    def load_column_selection():
        with open(join_path("sales_columns_selection.txt")) as f:
            return list(map(lambda t: t[1], [tuple(col.strip().split(', ')) for col in f.readlines()]))
        
    usecols = load_column_selection()
    
    csv_params['sep'] = ","
    csv_params['encoding'] = "ISO-8859-1"
#     csv_params['usecols'] = usecols
    
    sales = read_csv(filename, nrows, **csv_params)
    
    concat_date = lambda row: dt.datetime.combine(row['order_date'], row['order_time'])
    
    return (
        
        sales
        .dropna(how='all', axis=1)
        .loc[:, usecols]
        .assign(
            order_date=pd.to_datetime(sales['Order Date']).dt.date, 
            order_time=pd.to_datetime(sales['Order Time']).dt.time)
        .pipe(lambda df: df.assign(order_date=df.apply(concat_date, axis=1)))
        .drop(["Order Date", "Order Time", "order_time"], axis=1)
        .dropna(how='all', axis=1)
        .pipe(lambda df: df.where(~df.Description.isin(exclude)).dropna())
        .set_index("order_date")
        .groupby([pd.Grouper(freq=SAMPLING), SIZE_COLUMN] + list(map(lambda x: x[0], FILTERS)))
        .aggregate({"Quantity": 'sum', "Quantity Returned": 'sum'})
        .reset_index()
        .set_index("order_date")
        .rename(columns={SIZE_COLUMN: "Size"})
        .pipe(filter_sizes)
        
    )

def load_inventory(nrows=None):
    final_columns = []
    inventory = read_csv(join_path("merged_stocks.csv"), nrows, sep=",", encoding="latin-1")
    items = load_items()
    
    inventory = (
        inventory
        .where(~inventory["Size"].isin(('1', '1MT')))
        .dropna()
        .join(items, on="Item No_")
    )
    
    def convert_collection(s):
        try:
            return s.split("-")[0]
        except:
            return s
    
    return (
    
        inventory
        .assign(
            posting_date=pd.to_datetime(inventory["Posting Date"]), 
            Season=inventory["ACACollection"].map(convert_collection), 
            product_code=inventory["Merchandise Code"].combine_first(inventory["Product Group Code"]), 
            Size=inventory["Size"].str.replace("3XL", "XXXL"), 
            NetQuantity=inventory["Quantity"] + inventory["Sales"]
        )
# #         .where(inventory["Sales"] == 0)
# #         .dropna()
# #         .pipe(lambda df: df.where(df["Quantity"] != 0).dropna())
#         .drop(["Posting Date", "ACACollection", "Merchandise Code", "Product Group Code"], axis=1)
#         .rename(columns={"product_code": "Merchandise Code", "ACABrand": "Brand"})
#         .set_index("posting_date")
#         .pipe(filter_sizes)
    
    )

def load_items():
    selection = ['Item No_', 'Product Group Code', 'ACACollection', 'ACABrand']
    items = read_csv(join_path("ExportItemData_2_0_stripped.csv"), sep=",", encoding="latin-1")
    return (

        items[selection]
        .drop_duplicates(subset=selection[:2])
        .loc[~(items["ACACollection"].str.endswith('M', na=False))]
        .set_index('Item No_')
        
    )

join_path = lambda filename: os.path.join(DATA_DIR, filename)

def timeit(func, *args, **kwargs):
    t0 = time()
    out = func(*args, **kwargs)
    t1 = time()
    print(f"{func.__name__} took {t1 - t0: .2f} s.")
    return out

def read_csv(filename, nrows=None, **csv_params):
    if nrows is not None:
        csv_params["nrows"] = nrows
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        return pd.read_csv(filename, **csv_params)

def filter_sizes(data):
    # Select relevant sizes
    sizes = ['XXS', 'XS', 'S', 'M', 'L', 'XL', 'XXL', 'XXXL']
    column_name = "Size"
    categorical_size_type = CategoricalDtype(categories=sizes, ordered=True)
    data = data[data[column_name].isin(sizes)].copy()
    data[column_name] = data[column_name].astype(categorical_size_type)
    return data

sales = timeit(load_sales)
inventory = timeit(load_inventory)

KeyError: 'Item No_'

In [ ]:
def build_widgets():
    
    global sales  # Just to make sure from the start that sales is defined...
    
    date_defaults = [("start_date", START_DATE), ("end_date", END_DATE)]
    
    dropdown_names = FILTERS
    
    align_kw = dict(_css = (('.widget-label', 'min-width', '20ex'),), margin = '0px 0px 5px 12px')
    layout_kw = dict(width='50%',display='inline-flex',flex_flow='row wrap')
    
    # Extract unique values for dropdowns
    get_values = lambda data, column: [None] + sorted(data[column].unique())
    
    def build_dropdown(values, display_name):
        # Build a dropdown widget for interactivity
        return widgets.Dropdown(options=values, value=values[0], description=display_name, **align_kw)

    def build_date_picker(date_type, display_name, default):
        # Build a date picker widget for interactivity
        return widgets.DatePicker(description=display_name, value=default, **align_kw)
    
    widget_names = [[], [], []]
    
    date_picker = []
    clean = lambda s: s.replace("_", " ").title()
    for date_type, value in date_defaults:
        widget_names[0].append(date_type)
        date_picker.append(build_date_picker(date_type, clean(date_type), value))
    
    dropdowns = []
    for column, display_name in dropdown_names:
        widget_names[1].append(column)
        dropdowns.append(build_dropdown(get_values(sales, column), display_name))

    options = []
    
    widget_names[2].append("Shape")
    options.append(build_dropdown(["histogram", "kde"], "Plot"))
    
    widget_names[2].append("Relative")
    options.append(widgets.Checkbox(value=False, description="Relative", disabled=False))
    
    widget_names[2].append("Frequency")
    options.append(build_dropdown(['D', 'W', 'M', 'Q', 'Y'], "Frequency"))
    
    widget_names[2].append("y_data")
    options.append(build_dropdown([None] + list(map(lambda t: t[1], dropdown_names)), "Y-data"))
    
#     hboxes = [
#         HBox(layout=Layout(**layout_kw)),
#         HBox(layout=Layout(**layout_kw)),
#         HBox(layout=Layout(**layout_kw)),
#     ]
    
#     for i, array in enumerate([date_pickers, dropdowns]):     
#         hboxes[i].children = array

#     master_box = HBox(**layout_kw)
#     master_box.children = hboxes

    return list(zip(
        [name for l in widget_names for name in l], 
        [widget for l in [date_picker, dropdowns, options] for widget in l]  # master_box
    ))

In [ ]:
def prepare_data(data, **kwargs):
    assert isinstance(data, pd.DataFrame)
    data = data.copy()
    
    start_date = kwargs.get("start_date", START_DATE)
    end_date = kwargs.get("end_date", END_DATE)
    
    filters = [
    
        ("Web Shop Code", kwargs.get("retailer")),
        ("Brand", kwargs.get("brand")), 
        ("Season", kwargs.get("season")), 
        ("Merchandise Code", kwargs.get("product_code"))
    
    ]

    simple_filter = lambda data, column, value: data[data[column] == value].copy()

    def filter_date(data, start_date=None, end_date=None):
        # Filter start date and end date
        data = data.copy()
        convert_date = lambda date: dt.datetime(date.year, date.month, date.day)
        if start_date:
            data = data[data.index >= convert_date(start_date)]
        if end_date:
            data = data[data.index <= convert_date(end_date)]
        return data
    
    # Filter out unwanted values
    data = filter_date(data, start_date, end_date)
#     data = filter_sizes(data)
    
    for name, value in filters:
        if value is not None:
            data = simple_filter(data, name, value)
    
    return data

def prepare_size_data(data, **kwargs):
    
    data = prepare_data(data, **kwargs)
    relative = kwargs.get("relative", False)

    # Filter out unnecessary columns    
    data = data[["Quantity", "Size"]]
    
    # Group the data by sizes
    data = data.groupby("Size").sum()
    
    
    # Returns true quantities or proportions depending
    if not relative:
        return data
    
    proportion = lambda data: data / data.sum()
    return proportion(data)

In [ ]:
def plot_size_distribution(**kwargs):
    assert all([df_name in globals() for df_name in ("sales", "inventory")])
    assert "shape" in kwargs
    shape = kwargs["shape"]
    del kwargs["shape"]

    global sales
    global inventory
        
    sales_data = prepare_size_data(sales, **kwargs).rename(columns={"Quantity": "Out"})
    inventory_data = prepare_size_data(inventory, **kwargs).rename(columns={"Quantity": "In"})
    
    # Close the previous figure
    plt.close()
    
    fig = plt.figure(figsize=(8,4))
    ax = fig.add_subplot(111)
    
    data = pd.concat([inventory_data, sales_data], join='outer', axis=1).fillna(0)
#     display(data)
#     input()
        
    if shape != "histogram":
        data.plot.kde(ax=ax)
    if shape != "kde":
        data.plot.bar(ax=ax)
    
    format_chart(ax, kwargs)
      
def format_chart(ax, kwargs):
    
    kwargs = kwargs.copy()
    
    if "relative" in kwargs:
        del kwargs["relative"]
    
    def func_format(x, _):
        if x > 1: return f"{x:,.0f}"
        elif 0 < x < 1: return f"{x:.0%}"
        else: return ""
    
    format_key = lambda key: (
    
        (key.replace("_", " ").title() + "=")
        .replace("Start Date=", "From ")
        .replace("End Date=", "to ")    
    )
        
    def format_val(value):
        if isinstance(value, dt.date): return f"{value:%d-%m-%Y}"
        if isinstance(value, float): return f"{value:.0f}"
        return str(value)

        
    ax.set_xlabel("")
    [tl.set_rotation(0) for tl in ax.get_xticklabels()]
    
    ax.yaxis.set_major_formatter(FuncFormatter(func_format))
    
    cut_lines = lambda l: " ".join(l[:len(l) // 2] + ["\n"] + l[len(l)//2:])    
    title = " ".join([format_key(k) + format_val(v) for k, v in kwargs.items() if v])
    title = title if len(title) < 50 else cut_lines(title.split(" "))
    ax.set_title(title)
    plt.tight_layout()

## Shoeby sales: Sizes distribution

In [ ]:
ipywidgets = {k: v for k, v in build_widgets()}

size_dist_options = dict(
    
    shape=ipywidgets["Shape"],
    relative=ipywidgets["Relative"],
    start_date=ipywidgets["start_date"], 
    end_date=ipywidgets["end_date"],
    product_code=ipywidgets["Merchandise Code"],
    brand=ipywidgets["Brand"], 
    season=ipywidgets["Season"],
)

interactive(plot_size_distribution, **size_dist_options)

In [ ]:
def prepare_time_data(data, **kwargs):
    data = prepare_data(data, **kwargs)
    frequency = kwargs.get("frequency", SAMPLING)
    y_data = kwargs.get("y_data")
    if y_data:
        columns = FILTERS[list(map(lambda x: x[1], FILTERS)).index(y_data)][0]
        data = data.reset_index().pivot_table(index="order_date", columns=columns, values="Quantity", aggfunc="sum")
    else:
        data = data["Quantity"]
    return data.resample(frequency).sum()

In [ ]:
def plot_sales_history(**kwargs):
#     assert all([df_name in globals() for df_name in ("sales", "inventory")])
    global sales
    
    sales_data = prepare_time_data(sales, **kwargs)
    
#     pprint(sales_data)

    # Close the previous figure
    plt.close()
    
    fig = plt.figure(figsize=(8,4))
    ax = fig.add_subplot(111)
    
    sales_data.plot(ax=ax)
    format_chart(ax, kwargs)

In [ ]:
ipywidgets = {k: v for k, v in build_widgets()}

sales_history_options = dict(
    
    frequency=ipywidgets["Frequency"],
    y_data=ipywidgets["y_data"],
    start_date=ipywidgets["start_date"], 
    end_date=ipywidgets["end_date"],
    retailer=ipywidgets["Web Shop Code"],
    product_code=ipywidgets["Merchandise Code"],
    brand=ipywidgets["Brand"], 
    season=ipywidgets["Season"],
    
)

interactive(plot_sales_history, **sales_history_options)

In [ ]:
def prepare_inventory_data():
    return (
        inventory
        .loc[inventory["NetQuantity"] == 0]
        .groupby(["posting_date", "Item No_", "Size", "Color"])
        .agg({'Opening Inventory': 'max'})
        .reset_index()
        .set_index("posting_date")
    )

def plot_inventory(**kwargs):
    global opening_inventory
    data = opening_inventory.copy().rename(columns={"Item No_": "item_no"})
    for key, value in kwargs.items():
        if key in data.columns and value:
            print(key, value, end=" ")
            data = data.where(data[key] == value).dropna()
            print(data.shape)
#     if "sampling" in kwargs:
#         data = data.resample(kwargs["sampling"]).agg(lambda df: df.iloc[-1]["Opening Inventory"])
    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)
    sns.barplot(x="posting_date", y="Opening Inventory", hue="Size", data=data.reset_index().head(50), ax=ax)
    format_chart(ax)
    
def build_dropdown(values, display_name):
    # Build a dropdown widget for interactivity
    return widgets.Dropdown(options=values, value=values[0], description=display_name)

opening_inventory = prepare_inventory_data()
no_dd = build_dropdown(opening_inventory["Item No_"].unique(), "Item Number")
col_dd = build_dropdown([None] + list(opening_inventory["Color"].unique()), "Color")
dt_dd = build_dropdown(["D", "W", "M", "Q"], "Sampling")
interactive(plot_inventory, sampling=dt_dd, item_no=no_dd, Color=col_dd)